<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/inria-25-alpha.png" style="display:inline"></span><br/>

# Tâches et exceptions

[Animation](loop-stacks/index.html)

# bonnes pratiques

* attraper les exceptions
  * dans le code asynchrone

* autrement
  * lire les exceptions (appeler `Task.exception()`)

# Résumé

* une boucle = liste de tâches

* une tâche
  * une pile
  * un état (terminé ?)
  * un résultat ou une exception